In [91]:
import pandas as pd
import os
import requests
import time

from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o", temperature=0)
from langchain_openai import OpenAIEmbeddings
embedding=OpenAIEmbeddings(model="text-embedding-3-small")
from langchain_community.vectorstores import FAISS

from pdf2image import convert_from_path
poppler_path =".\\poppler\\Library\\bin"
import pytesseract
pytesseract.pytesseract.tesseract_cmd = (r"C:\Program Files\Tesseract-OCR\tesseract.exe")
from PIL import Image
Image.MAX_IMAGE_PIXELS = 1000000000

import pymysql
from bs4 import BeautifulSoup

In [71]:
df=pd.read_csv("listlist.csv")

In [72]:
class url(BaseModel):
    Year:str=Field(description="Common Data Set year for this college, format like '2023-2024'")
    URL:str=Field(description="Common Data Set url for this college, needs to be full url")
class urls(BaseModel):
    URLs:list[url]=Field(description="array of Year and URL")

In [73]:
URL_structured_output=llm.with_structured_output(urls)
prompt = ChatPromptTemplate.from_messages([
    ("system", "based on the raw html blow, extract the {college} common dataset urls, list all urls but only this college's urls, the url must be exactly same as in the html, including lowercase and uppercase letters, if nothing found, return empty"),
    ("human", "html：{html}"),
])
URL_chain=prompt|URL_structured_output

------------------------------download pdfs--------------------------------------

<span style="color:red;font-size:20px">----------------------Change row_number here-----------------</span>

In [125]:
row_number=16
path="PDFs2"
row=df.iloc[row_number]
college_name=row['name']
college_id=row['postid']
college_unitid=row['unitid']
college_path=os.path.join(path,str(college_id))

from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

os.mkdir(college_path)

url=row['url0']
if url[-3:]=='pdf':
    url=row['url1']
if url[-3:]=='pdf':
    url=row['url2']
if url[-3:]=='pdf':
    raise Exception("All 3 urls are pdf links")

html=requests.get(url,verify=False,headers = {'User-agent': 'my personal bot 0.1'},timeout=10).content
soup=BeautifulSoup(html,"html.parser")
for x in soup(['style','script']):
    x.decompose()
html=str(soup)
response=URL_chain.invoke({"college":college_name,"html":html[:int(-len(html)/10)]})

for item in response.URLs:
    year=item.Year
    pdf_url=item.URL
    file_response=requests.get(pdf_url,verify=False,headers = {'User-agent': 'my personal bot 0.1'},timeout=10)
    if "Content-Type" in file_response.headers and file_response.headers["Content-Type"]=="application/pdf":
        file_path=os.path.join(college_path,year+".pdf")
        with open(file_path,'wb') as file:
            file.write(file_response.content)
    if "Content-Type" in file_response.headers and file_response.headers["Content-Type"].startswith("text/html"):
        file_path=os.path.join(college_path,year+".html")
        with open(file_path,'wb') as file:
            file.write(file_response.content)
    print(college_id,college_name,year,"done!")
    time.sleep(2)
print("All done!!!")

8556 Rice University 2022-23 done!
8556 Rice University 2021-22 done!
8556 Rice University 2020-21 done!
8556 Rice University 2019-20 done!
8556 Rice University 2018-19 done!
8556 Rice University 2017-18 done!
8556 Rice University 2016-17 done!
8556 Rice University 2015-16 done!
8556 Rice University 2014-15 done!
8556 Rice University 2013-14 done!
8556 Rice University 2012-13 done!
8556 Rice University 2011-12 done!
8556 Rice University 2010-11 done!
8556 Rice University 2009-10 done!
8556 Rice University 2008-09 done!
All done!!!


----------------------------------extract information from pdfs----------------------------------

In [119]:
class GPA_info(BaseModel):
    Name:str=Field(description='name of the college or University')
    Address:str=Field(description='address of the college or University')
    Year:str=Field(description="Year of Common Data Set, right after 'Common Data Set'")
    GPA400:str=Field(description="Percent who had GPA of 4.0, percentage number right afte percent who had GPA of 4.0")
    GPA375:str=Field(description="Percent who had GPA between 3.75 and 3.99, percentage number right afte 'Percent who had GPA between 3.75 and 3.99'")
    GPA350:str=Field(description="Percent who had GPA between 3.50 and 3.74, percentage number right afte 'Percent who had GPA between 3.50 and 3.74'")
    GPA325:str=Field(description="Percent who had GPA between 3.25 and 3.49, percentage number right afte 'Percent who had GPA between 3.25 and 3.49'")
    GPA300:str=Field(description="Percent who had GPA between 3.00 and 3.24, percentage number right afte 'Percent who had GPA between 3.00 and 3.24'")
    GPA250:str=Field(description="Percent who had GPA between 2.50 and 2.99, percentage number right afte 'Percent who had GPA between 2.50 and 2.99'")
    GPA200:str=Field(description="Percent who had GPA between 2.0 and 2.49, percentage number right afte 'Percent who had GPA between 2.0 and 2.49'")
    GPA100:str=Field(description="Percent who had GPA between 1.0 and 1.99, percentage number right afte 'Percent who had GPA between 1.0 and 1.99'")
    GPA000:str=Field(description="Percent who had GPA below 1.0, percentage number right afte 'Percent who had GPA below 1.0'")

GPA_info_structured_output=llm.with_structured_output(GPA_info)
prompt = ChatPromptTemplate.from_messages([
    ("system", "Only based on documents content below, ouput information strictly based on format, output percentage number should not include the percetage symbol, do not make up numbers, if you cannot find information in the documents, output all zeros"),
    ("human", "documents：{documents}"),
])
GPA_chain=prompt | GPA_info_structured_output

path="PDFs2"
pdf_path=os.path.join(path,str(college_id))
pdfs=os.scandir(pdf_path)
GPA_df=pd.DataFrame(columns=['postid','Name','ExtractName','Address','Year','ExtractYear','GPA400','GPA375','GPA350','GPA325','GPA300','GPA250','GPA200','GPA100','GPA000'])
for pdf in pdfs:
    year=pdf.name[:-4]
    img_path=os.path.join(pdf_path,year)
    pages=convert_from_path(pdf.path,poppler_path=poppler_path)
    os.mkdir(img_path)
    for index,page in enumerate(pages):
        img_file_path=os.path.join(img_path,str(index)+".jpg")
        page.save(img_file_path)
        text = str(pytesseract.image_to_string(page,config="--psm 11"))
        if index==0:
            faiss_index=FAISS.from_texts([text], embedding)
        else:
            faiss_index.add_texts([text])
    docs_info=faiss_index.similarity_search("address information", k=2)
    docs_GPA = faiss_index.similarity_search("GPA", k=4)
    docs=docs_GPA+docs_info
    response=GPA_chain.invoke({"documents":docs})
    GPA_df2=pd.DataFrame([[
        college_id,
        college_name,
        response.Name,
        response.Address,
        year,
        response.Year,
        response.GPA400,
        response.GPA375,
        response.GPA350,
        response.GPA325,
        response.GPA300,
        response.GPA250,
        response.GPA200,
        response.GPA100,
        response.GPA000
    ]],columns=['postid','Name','ExtractName','Address','Year','ExtractYear','GPA400','GPA375','GPA350','GPA325','GPA300','GPA250','GPA200','GPA100','GPA000'])
    GPA_df=pd.concat([GPA_df,GPA_df2]).reset_index(drop=True)
pdfs.close()
GPA_df

,postid,Name,ExtractName,Address,Year,ExtractYear,GPA400,GPA375,GPA350,GPA325,GPA300,GPA250,GPA200,GPA100,GPA000
0,8693,University of California-Berkeley,"University of California, Berkeley","Berkeley, CA",2020-2021,2017-2018,31.5,51.7,9.4,2.3,2.5,2.6,0.0,0.0,0.0
1,8693,University of California-Berkeley,"University of California, Berkeley","Berkeley, CA",2021-2022,2021-2022,33.0,50.4,12.9,2.7,0.7,2,0.1,0,0
2,8693,University of California-Berkeley,Unknown,Unknown,2022-2023,2021-2022,38.8,51.8,8.0,1.0,4,0.1,0,0,0
3,8693,University of California-Berkeley,UC Berkeley,"Berkeley, CA",2023-2024,2024,0,0,0,0,0,0,0,0,0


-------------------------------------import data into MySQL server----------------------------------------

In [123]:
connection=pymysql.connect(
    db     = os.environ['db_name'],
    user   = os.environ['db_user'],
    passwd = os.environ['db_pass'],
    host   = os.environ['db_host'],
    port   = 3306,
)
cursor = connection.cursor()
for index,row in GPA_df.iterrows():
    GPA_year=row['Year']
    query="""INSERT INTO `GPA2` (`post_id`, `unitid`,`Name`, `Year`, `GPA400`, `GPA375`, `GPA350`, `GPA325`, `GPA300`, `GPA250`, `GPA200`, `GPA100`, `GPA000`) 
            VALUES ({},{},'{}','{}',{},{},{},{},{},{},{},{},{})""".format(college_id,college_unitid,college_name,GPA_year,row['GPA400'],row['GPA375'],row['GPA350'],row['GPA325'],row['GPA300'],row['GPA250'],row['GPA200'],row['GPA100'],row['GPA000'])
    cursor.execute(query)
    #result = cursor.fetchall()
    connection.commit()
cursor.close()
connection.close()